# Installation

## Guides

* [Get Started](https://blog.alexellis.io/getting-started-with-docker-on-raspberry-pi/)
* [How to Docker](https://jonnylangefeld.github.io/learning/Docker/How%2Bto%2BDocker.html)
* [Docker Hub](https://hub.docker.com/)


## Process

1. Install on Jessie Lite.  Setup ssh.
2. Change memory allocations: Edit /boot/config.txt and add this line: gpu_mem=16  
3. Install Docker:  `curl -sSL https://get.docker.com | sh`

4. Set Docker to auto-start:  `sudo systemctl enable docker`
5. Start Docker With `sudo systemctl start docker`
6. The Docker client can only be used by root or members of the docker group. Add pi or your equivalent user to the docker group: `sudo usermod -aG docker pi`
7. Restart

## Start Alpine

1. `docker run -ti arm32v6/alpine:3.5 /bin/sh`
2. `exit`
3. Spit out the date from the Alpine image: `docker run arm32v6/alpine:3.5 date`

## Build a New Image Called curl_docker

You can build an image using Docker, `resin/rpi-raspbian`, Raspbian as the base.

1.  Download the latest image, this gets the latest docker image:  `docker run -ti resin/rpi-raspbian:latest /bin/sh`
2.  Create a file called "Docker" with this in it:

```
FROM resin/rpi-raspbian:latest  
ENTRYPOINT []

RUN apt-get update && \  
    apt-get -qy install curl ca-certificates

CMD ["curl", "https://docker.com"] 
```

Build the image:
`docker build -t curl_docker .`
Run the image:
`docker run curl_docker`

## Use GPIO

### Create a Base Image
Require an additional flag at runtime of `--privileged`

1.  Download the latest image.  
2.  Create a file called "Docker" with this in it:

```
FROM resin/rpi-raspbian:latest  
ENTRYPOINT []

RUN apt-get -q update && \  
    apt-get -qy install \
        python python-pip \
        python-dev python-pip gcc make  

RUN pip install rpi.gpio  
```

3.  Build the base image `docker build -t gpio-base .`

### Create a Python Script app.py

```
import RPi.GPIO as GPIO  
import time  
GPIO.setmode(GPIO.BCM)  
led_pin = 17  
GPIO.setup(led_pin, GPIO.OUT)

while(True):  
    GPIO.output(led_pin, GPIO.HIGH)
    time.sleep(1)
    GPIO.output(led_pin, GPIO.LOW)
    time.sleep(1)
```

### Create a New Image off the Base Image

1.  Just use ADD to transfer the script into a new image depriving from gpio-base.  We will build a new Docker image from the gpio-base image.  Create a Docker file:

```
FROM gpio-base:latest  
ADD ./app.py ./app.py

CMD ["python", "app.py"] 
```

2.  Build the image: `docker build -t blink .`

3.  Run in privelaged mode: `docker run -ti --privileged blink`


## Random Important Commands
* `docker images` - List images you have built.
* `docker ps -a` - List working images and their status.
* ```docker rm $(docker ps -a -f status=exited -q)``` - deletes all open containers in Docker.